<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5f032ae7ff2a2b8cd57bbe75ef1acd8646be887d45a2ce3dcf08d1389dc197a1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 15:01:42
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 70.94 K (0.5%)
Current PnL: -19.73 L (-13.14%)
CY Booked + Current PnL: -8.29 L (-5.52%)
-------------------
Total profit:  1.85 L
Total loss:  -21.58 L
-------------------
Total Booked + Current PnL: 18.69 L (15.18%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.06%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.23 L (61.49%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.53%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119054.0,18550.0,10560.0,-2.14,18.46,8.87,28.96,79.0,1.76,0.84,58.69,MH,ATH,METALS
14,BLUESTARCO,2080.00,13.00,59.0,X-MC,3.22,195000.0,30330.0,13006.0,3.05,18.42,6.67,26.31,89.0,2.33,1.38,27.87,X40N,NTT,AC
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.67,213693.0,9193.0,14104.0,3.09,4.50,6.60,11.39,10.0,0.65,1.51,13.85,X40N,ATH,PHARMA
84,VOLTAS,1530.00,1.72,62.0,X-MC,1.10,213300.0,21558.0,16189.0,2.89,11.24,7.59,19.69,99.0,1.33,1.51,18.80,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.01,83205.0,-17572.0,17664.0,-0.30,-17.44,21.23,0.09,245.0,-0.99,0.59,9.31,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,1006.00,-39.60,52.0,M-SC,13.66,75154.0,-26005.0,100894.0,5.97,-25.71,134.25,74.03,202.0,-0.26,0.53,5.97,AR,BTT,AUTO
46,KPIGREEN,731.05,-6.33,41.0,H-SC,6.22,111712.0,-13585.0,72512.0,5.10,-10.84,64.91,47.03,141.0,-0.19,0.79,39.87,MH,ATH,POWER
10,BANDHANBNK,400.00,-9.22,60.0,H-SC,2.94,227800.0,-50759.0,308191.0,4.10,-18.22,135.29,92.42,151.0,-0.16,1.61,31.65,XY24,NTT,BANKS
63,SFL,1287.00,33.18,48.0,M-SC,7.19,175814.0,-86404.0,156228.0,3.52,-32.95,88.86,26.63,229.0,-0.55,1.24,9.31,XY24,NTT,MISC
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,50.0,M-MC,12.65,227097.0,2135.0,165508.0,-3.36,0.95,72.88,74.53,192.0,0.01,1.61,33.51,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-24.05,46.0,H-MC,6.64,102809.0,-29026.0,71247.0,-2.91,-22.02,69.30,32.03,98.0,-0.41,0.73,15.15,OX40N,NTT,IT
32,HINDZINC,730.22,33.63,66.0,M-LC,7.82,217826.0,12750.0,99830.0,-2.25,6.22,45.83,54.89,52.0,0.13,1.54,31.81,X5K,ATH,METALS
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119054.0,18550.0,10560.0,-2.14,18.46,8.87,28.96,79.0,1.76,0.84,58.69,MH,ATH,METALS
82,VALIANTORG,838.00,-291.52,30.0,H-SC,1.68,106420.0,-57185.0,178498.0,-1.93,-34.95,167.73,74.15,139.0,-0.32,0.75,36.41,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,50.0,M-MC,12.65,227097.0,2135.0,165508.0,-3.36,0.95,72.88,74.53,192.0,0.01,1.61,33.51,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.79,131598.0,-10881.0,30399.0,1.10,-7.64,23.10,13.70,163.0,-0.36,0.93,55.32,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,52.0,H-SC,1.95,222822.0,-46845.0,83179.0,0.91,-17.37,37.33,13.47,138.0,-0.56,1.58,13.30,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.01,83205.0,-17572.0,17664.0,-0.30,-17.44,21.23,0.09,245.0,-0.99,0.59,9.31,OX40N,NTT,DURABLES
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137390.0,-38513.0,80538.0,-0.82,-21.89,58.62,23.89,149.0,-0.48,0.97,18.65,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.05,46.0,H-MC,6.64,102809.0,-29026.0,71247.0,-2.91,-22.02,69.30,32.03,98.0,-0.41,0.73,15.15,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,50.0,M-MC,12.65,227097.0,2135.0,165508.0,-3.36,0.95,72.88,74.53,192.0,0.01,1.61,33.51,XY24,BTT,STEEL
32,HINDZINC,730.22,33.63,66.0,M-LC,7.82,217826.0,12750.0,99830.0,-2.25,6.22,45.83,54.89,52.0,0.13,1.54,31.81,X5K,ATH,METALS
79,UNIONBANK,163.00,-11.63,57.0,M-LC,9.18,158373.0,17533.0,26955.0,1.06,12.45,17.02,31.59,66.0,0.65,1.12,42.23,XY24,NTT,BANKS
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152768.0,1823.0,107625.0,0.87,1.21,70.45,72.51,53.0,0.02,1.08,7.30,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.95,195403.0,15595.0,96822.0,1.00,8.67,49.55,62.52,88.0,0.16,1.38,45.72,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119054.0,18550.0,10560.0,-2.14,18.46,8.87,28.96,79.0,1.76,0.84,58.69,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.95,195403.0,15595.0,96822.0,1.00,8.67,49.55,62.52,88.0,0.16,1.38,45.72,XR,NTT,BANKS
32,HINDZINC,730.22,33.63,66.0,M-LC,7.82,217826.0,12750.0,99830.0,-2.25,6.22,45.83,54.89,52.0,0.13,1.54,31.81,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.95,127847.0,4511.0,127118.0,-1.04,3.66,99.43,106.72,119.0,0.04,0.90,27.89,AR,ATH,MISC
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152768.0,1823.0,107625.0,0.87,1.21,70.45,72.51,53.0,0.02,1.08,7.30,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.55,206518.0,-45250.0,98468.0,0.44,-17.97,47.68,21.13,27.0,-0.46,1.46,11.09,X40,ATH,PAINTS
23,DABUR,735.00,-12.30,36.0,X-MC,3.75,237480.0,-10588.0,119001.0,0.88,-4.27,50.11,43.70,102.0,-0.09,1.68,9.56,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
56,RELAXO,1176.00,-39.97,38.0,X-SC,2.55,81818.0,-63342.0,139271.0,-0.62,-43.64,170.22,52.31,136.0,-0.45,0.58,9.52,X40N,NTT,FOOTWEAR
22,COLPAL,3767.14,-4.76,39.0,X-MC,7.27,222540.0,-40825.0,154176.0,0.76,-15.50,69.28,43.04,84.0,-0.26,1.58,3.31,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-5.75,40.0,X-MC,0.56,233766.0,-13965.0,89018.0,0.85,-5.64,38.08,30.30,92.0,-0.16,1.65,6.91,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,1.72,62.0,X-MC,1.10,213300.0,21558.0,16189.0,2.89,11.24,7.59,19.69,99.0,1.33,1.51,18.80,XY25,NTT,AC
65,SIEMENS,4671.50,-1.97,47.0,H-LC,1.66,157650.0,-28445.0,75924.0,-0.80,-15.29,48.16,25.51,15.0,-0.37,1.12,16.18,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-19.88,43.0,X-MC,1.79,137625.0,1492.0,29631.0,-1.22,1.10,21.53,22.86,91.0,0.05,0.97,13.85,X40,ATH,INSURANCE
73,TATAMOTORS,1065.00,-21.50,44.0,X-LC,1.89,247239.0,-29284.0,139344.0,-0.07,-10.59,56.36,39.81,54.0,-0.21,1.75,18.49,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.25,39.0,X-LC,10.35,283591.0,-32251.0,144915.0,-0.31,-10.21,51.10,35.67,5.0,-0.22,2.01,2.23,X40N,ATH,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
22,COLPAL,3767.14,-4.76,39.0,X-MC,7.27,222540.0,-40825.0,154176.0,0.76,-15.50,69.28,43.04,84.0,-0.26,1.58,3.31,XY25,ATH,FMCG
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196920.0,-3218.0,24103.0,0.70,-1.61,12.24,10.44,4.0,-0.13,1.39,3.87,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.48,39.0,X-MC,4.96,180367.0,-20139.0,117293.0,-0.10,-10.04,65.03,48.45,90.0,-0.17,1.28,4.01,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53764.0,-11242.0,162168.0,3.15,-17.29,301.63,232.18,198.0,-0.07,0.38,3.15,XY24,NTT,MISC
56,RELAXO,1176.00,-39.97,38.0,X-SC,2.55,81818.0,-63342.0,139271.0,-0.62,-43.64,170.22,52.31,136.0,-0.45,0.58,9.52,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.29,42.0,X-SC,4.43,96828.0,-31842.0,77143.0,-0.49,-24.75,79.67,35.20,219.0,-0.41,0.69,11.14,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.94,51.0,X-SC,1.45,108052.0,6048.0,37872.0,0.73,5.93,35.05,43.06,122.0,0.16,0.76,24.06,X40N,ATH,MISC
33,HONAUT,58357.33,-22.26,49.0,X-SC,2.12,109545.0,-18393.0,65530.0,0.76,-14.38,59.82,36.84,143.0,-0.28,0.78,12.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,50.0,X-LC,12.17,284331.0,-61625.0,129854.0,-1.21,-17.81,45.67,19.73,1.0,-0.47,2.01,4.72,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,322269.0,9381.0,162295.0,0.25,3.00,50.36,54.87,3.0,0.06,2.28,9.85,X40,BTT,IT
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196920.0,-3218.0,24103.0,0.70,-1.61,12.24,10.44,4.0,-0.13,1.39,3.87,X40,NTT,FMCG
83,VBL,671.64,-21.25,39.0,X-LC,10.35,283591.0,-32251.0,144915.0,-0.31,-10.21,51.10,35.67,5.0,-0.22,2.01,2.23,X40N,ATH,FMCG
1,ABB,7934.00,-40.94,42.0,H-LC,7.52,247032.0,-14587.0,133793.0,0.35,-5.58,54.16,45.57,7.0,-0.11,1.75,5.11,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,54.0,L-SC,29.23,83597.0,-29952.0,70004.0,1.97,-26.38,83.74,35.27,268.0,-0.43,0.59,104.61,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,45.0,H-SC,10.41,91440.0,-6540.0,28145.0,1.20,-6.67,30.78,22.05,152.0,-0.23,0.65,32.91,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.28,119054.0,18550.0,10560.0,-2.14,18.46,8.87,28.96,79.0,1.76,0.84,58.69,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.79,131598.0,-10881.0,30399.0,1.10,-7.64,23.10,13.70,163.0,-0.36,0.93,55.32,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,66.0,M-SC,15.39,146250.0,-2970.0,147157.0,0.89,-1.99,100.62,96.62,208.0,-0.02,1.04,58.27,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,67.0,H-SC,9.08,154428.0,-24643.0,90371.0,1.88,-13.76,58.52,36.71,135.0,-0.27,1.09,53.22,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,66.0,M-SC,15.39,146250.0,-2970.0,147157.0,0.89,-1.99,100.62,96.62,208.0,-0.02,1.04,58.27,XY25,NTT,FINANCE
32,HINDZINC,730.22,33.63,66.0,M-LC,7.82,217826.0,12750.0,99830.0,-2.25,6.22,45.83,54.89,52.0,0.13,1.54,31.81,X5K,ATH,METALS
26,GREENPANEL,537.00,239.15,62.0,M-SC,4.73,151353.0,-31725.0,110700.0,1.67,-17.33,73.14,43.14,230.0,-0.29,1.07,42.37,XY24,NTT,MISC
68,STARHEALTH,761.00,17.10,68.0,H-SC,7.43,260875.0,-20248.0,153108.0,0.20,-7.20,58.69,47.26,144.0,-0.13,1.85,39.77,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.44
2,50,75.20


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.03
LC    32.24
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.00
X40      14.59
X40N     11.27
XR       10.97
XY25     10.44
AR        8.12
OX40N     7.60
X200      1.74
MH        1.63
X5K       1.54
SR        1.10
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.50
X-LC    22.89
X-MC    17.21
M-SC    12.55
H-LC     4.61
H-MC     4.60
X-SC     4.53
M-LC     3.74
M-MC     1.61
L-SC     1.45
L-LC     1.00
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-5.55,40.34
IT,12.80,-16.18,79.10
FINANCE,10.51,-13.77,68.71
MISC,6.93,-17.02,81.86
BANKS,6.44,-9.53,67.15
PAINTS,5.49,-18.95,36.68
ELECTRICAL,5.30,-10.08,48.70
INSURANCE,3.78,-3.64,42.38
AC,3.58,4.19,11.85


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3194076.0,22
XR,1359070.0,14
AR,1182336.0,9
X40,777582.0,10
X40N,644337.0,9
OX40N,574534.0,10
XY25,464022.0,7
SR,265372.0,2
X5K,99830.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3394948.0,25
M-SC,1269479.0,15
X-MC,1103891.0,12
X-LC,1007554.0,13
X-SC,541473.0,6
H-MC,358931.0,4
H-LC,288834.0,3
L-SC,256980.0,3
M-LC,234410.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1186081.0      6
           AR         807052.0      5
           XR         796118.0      7
M-SC       XY24       678818.0      6
X-MC       XY24       587252.0      4
X-LC       X40        484464.0      5
X-SC       X40N       302162.0      4
H-SC       OX40N      267813.0      4
           SR         265372.0      2
X-MC       X40        215975.0      4
X-LC       X40N       211876.0      3
H-LC       AR         209717.0      2
X-LC       XY24       206992.0      2
H-MC       XY24       180302.0      1
X-MC       XY25       170365.0      2
L-SC       XR         168456.0      2
M-SC       AR         165567.0      2
M-MC       XY24       165508.0      1
X-SC       XY24       162168.0      1
M-SC       XY25       147157.0      1
           OX40N      146950.0      4
           XR         130987.0      2
X-MC       X40N       130299.0      2
M-LC       XR         107625.0      1
X-LC       XY25       104222.0      3
M-LC       X5K         99830.0      1
H-MC       XR          96822.0      1
L-SC       OX40N       88524.0      1
H-LC       X200        79117.0      1
X-SC       X40         77143.0      1
H-SC       MH          72512.0      1
H-MC       OX40N       71247.0      1
L-MC       XR          59062.0      1
L-LC       XY25        42278.0      1
M-LC       XY24        26955.0      1
H-MC       MH          10560.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
